In [1]:
import numpy as np
from fuzzywuzzy import process
from fuzzywuzzy import fuzz


def find_row(df, column, row_name):
    rows = df[column].values
    #print(process.extract(row_name, rows, scorer=fuzz.token_sort_ratio))
    return df.loc[df[column]==process.extract(row_name, rows, scorer=fuzz.token_sort_ratio)[0][0],:]

import pandas as pd

xl = pd.ExcelFile("DH_nagy.xlsx")

sheet_list = xl.sheet_names

print(sheet_list)

pair_list = list(pd.read_excel("DH_nagy.xlsx", sheet_name = sheet_list[0]).groupby("NÉV")["VIZSGÁLATI AZONOSÍTÓ"].apply(list))
pair_list = [x for x in pair_list if len(x)>1]
pair_list

C:\ProgramData\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


['PÁCIENSEK', 'ADATOK', 'DH01', 'DH02', 'DH03', 'DH04', 'DH05', 'DH06', 'DH07', 'DH08', 'DH09', 'DH10', 'DH11', 'DH12', 'DH13', 'DH14', 'DH15', 'DH16', 'DH17', 'DH18', 'DH19', 'DH20', 'DH21', 'DH22', 'DH23', 'DH24', 'DH25', 'DH26', 'DH27', 'DH28', 'DH29', 'DH30', 'DH31']


[['DH15', 'DH18'],
 ['DH13', 'DH22'],
 ['DH20', 'DH23'],
 ['DH03', 'DH09'],
 ['DH02', 'DH11'],
 ['DH01', 'DH04'],
 ['DH14', 'DH21'],
 ['DH05', 'DH12'],
 ['DH06', 'DH08'],
 ['DH16', 'DH19'],
 ['DH07', 'DH10'],
 ['DH26', 'DH27']]

In [2]:
def df_values_pairs_hosszu(file = "DH_nagy.xlsx", nevek = pair_list, val = "mOsm (mmol/kg)"):
    df_hosszu = pd.read_excel("DH_nagy.xlsx", sheet_name = nevek[0][1]).iloc[:,:14]
    df_hosszu = find_row(df_hosszu, column = 'Mintavételi sorszám', row_name = val)
    for par in nevek[1:]:
        df_hosszu_ = pd.read_excel("DH_nagy.xlsx", sheet_name = par[1]).iloc[:,:14]
        df_hosszu_ = find_row(df_hosszu_, column = 'Mintavételi sorszám', row_name = val)
        df_hosszu = df_hosszu.append(df_hosszu_.iloc[-1,:]) 
    df_hosszu.columns = ['Minta', 'Nyugalmi (T. előtt)', "20'", "40'",
       "60'", "80'", "100'", "120'", 'VM', "R5'", "R30'", "nan1", 'nan2',
       "nan3", "nan4", "nan5"]
    df_hosszu["Minta"] = [i[1] for i in pair_list]
    return df_hosszu 

In [3]:
def df_values_pairs_rovid(file = "DH_nagy.xlsx", nevek = pair_list, val = "mOsm (mmol/kg)"):
    df_rov = pd.read_excel("DH_nagy.xlsx", sheet_name = nevek[0][0]).iloc[:,:13]
    df_rov = find_row(df_rov, column = 'Mintavételi sorszám', row_name = val)
    for par in nevek[1:]:
        #print(par)
        df_rov_ = pd.read_excel("DH_nagy.xlsx", sheet_name = par[0]).iloc[:,:13]
        df_rov_ = find_row(df_rov_, column = 'Mintavételi sorszám', row_name = val)
        df_rov = df_rov.append(df_rov_.iloc[-1,:]) 
    df_rov.columns = ['Minta', 'Nyugalmi (T. előtt)', 'RQ 0,9', 'RQ 1,0', 'Terhelés Után Max (TU)', "R1'", "R2'", "R3'", "R4'", "R5'", "R7'", "R10'", "R30'"]
    df_rov["Minta"] = [i[0] for i in pair_list]
    return df_rov
#list(pd.read_excel("DH_nagy.xlsx", sheet_name = pair_list[0][0]).iloc[0,:13])


In [4]:
df_hgb = df_values_pairs_rovid(file = "DH_nagy.xlsx", nevek = pair_list, val = "tHb (g/dL)")
df_hct = df_values_pairs_rovid(file = "DH_nagy.xlsx", nevek = pair_list, val = "Hct (%)")

df_hgb

,Minta,Nyugalmi (T. előtt),"RQ 0,9","RQ 1,0",Terhelés Után Max (TU),R1',R2',R3',R4',R5',R7',R10',R30'
77,DH15,15.1,15.7,15.8,16.8,17.2,17.2,17.3,16.5,17,16.4,14.9,15.3
77,DH13,15.2,15.3,15.8,17.1,16.6,16.8,16.6,16.3,16.3,16.5,16.1,15.6
80,DH20,15.8,16.7,16.7,18,17.4,17.6,17.2,17.5,17.3,17.3,17,16.1
110,DH03,15.4,15.7,16.7,17.9,17.7,17.6,18,NaN,17.1,NaN,NaN,15.8
76,DH02,15.4,15.2,15.8,13.7,15.8,16.3,15.3,16,14.6,16.1,14.3,15
76,DH01,14.9,13.1,16.9,15.5,NaN,14.7,NaN,14.7,NaN,16.1,15.2,13.9
77,DH14,17.1,17.3,17,17.5,18.8,18.6,18.4,18.6,18.3,18.3,17.9,16.7
77,DH05,16,15.8,16,16.8,17.5,16.2,17,17,16.6,16.1,16.2,16
77,DH06,16.1,15.7,16.6,17.4,17.7,16.9,17.4,17.4,17.2,17,16.8,16
77,DH16,16.3,16.5,16.5,17.9,18,18,17.9,17.5,17.6,17.2,16.8,15.6


In [5]:
df_hct

,Minta,Nyugalmi (T. előtt),"RQ 0,9","RQ 1,0",Terhelés Után Max (TU),R1',R2',R3',R4',R5',R7',R10',R30'
97,DH15,46.3,48,48.6,51.5,52.8,52.6,53,50.5,52,50.2,45.7,46.9
97,DH13,46.7,46.8,48.5,52.5,51,51.5,50.9,50.1,49.8,50.6,49.4,47.9
100,DH20,48.5,51.3,51.2,55.2,53.4,53.9,52.6,53.7,53.1,53.1,52.1,49.2
130,DH03,47.3,48.2,51.2,54.8,54.2,53.8,55.1,NaN,52.4,NaN,NaN,48.4
96,DH02,47.1,46.5,48.3,42.1,48.5,50,47,49,44.8,49.5,43.8,46.1
96,DH01,45.6,40.2,51.8,47.6,NaN,44.9,NaN,44.9,NaN,49.4,46.7,42.7
97,DH14,52.4,53,52.1,53.7,57.7,57,56.3,57,56,56.1,54.8,51.2
97,DH05,49.1,48.3,49.1,51.6,53.7,49.8,52.1,52.1,51,49.4,49.5,49.2
97,DH06,49.3,48.2,50.8,53.5,54.2,51.9,53.2,53.4,52.7,52.1,51.4,49
97,DH16,50.1,50.7,50.6,54.8,55.1,55.1,54.8,53.6,53.9,52.8,51.5,47.9


In [6]:
import plotly.express as px

df_hgb.columns = ['Minta', 'Rest', 'RER 0,9', 'RER 1,0',
       'Max', "R1'", "R2'", "R3'", "R4'", "R5'", "R7'",
       "R10'", "R30'"]
df_hct.columns = ['Minta', 'Rest', 'RER 0,9', 'RER 1,0',
       'Max', "R1'", "R2'", "R3'", "R4'", "R5'", "R7'",
       "R10'", "R30'"]

df_hct_long = df_hct.melt()
df_hct_long = df_hct_long.loc[df_hct_long.variable != "Minta",:]

df_hgb_long = df_hgb.melt()
df_hgb_long = df_hgb_long.loc[df_hgb_long.variable != "Minta",:]

fig = px.box(df_hct_long, x="variable", y="value")
fig.show()

In [7]:
y_hct = df_hct.iloc[:,1:].fillna(df_hct.iloc[:,1:].median()).apply(lambda x: np.median(x), axis=0)
x_hct = df_hct.iloc[:,1:].columns
y_hgb = df_hgb.iloc[:,1:].fillna(df_hgb.iloc[:,1:].median()).apply(lambda x: np.median(x), axis=0)
x_hgb = df_hgb.iloc[:,1:].columns


In [8]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from itertools import cycle

df_hct_long = df_hct_long.dropna()

palette = cycle(px.colors.qualitative.Antique)

fig = make_subplots(rows=2, cols=1, subplot_titles=("Hemoglobin [g/dl]", "Hematocrit [%]"))

fig.add_trace(go.Scatter(
    x=x_hgb, y=y_hgb,
    name='HGB line', marker_color=next(palette)), 
    row = 1, col =1
)
fig.add_trace(go.Box(
    x=df_hgb_long.variable, y=df_hgb_long.value,
    name='HGB box', marker_color=next(palette)), 
    row = 1, col =1
)


fig.add_trace(go.Scatter(
    x=x_hct, y=y_hct,
    name='HCT line', marker_color=next(palette)), 
    row = 2, col =1
)
fig.add_trace(go.Box(
    x=df_hct_long.variable, y=df_hct_long.value,
    name='HCT box', marker_color=next(palette)), 
    row = 2, col =1
)
#fig.layout.xaxis.titlefont.family = 'Times New Roman'
#fig.layout.yaxis.titlefont.family = 'Times New Roman'
fig.update_layout(font_family='Times New Roman', showlegend=False)
fig.update_layout(height=600, width=800, title_text="")
fig.show()
#box(df_hct_long, x="variable", y="value")

In [17]:
df_mosm_hosszu = df_values_pairs_hosszu(file = "DH_nagy.xlsx", nevek = pair_list, val = "mOsm (mmol/kg)")
df_hgb_hosszu = df_values_pairs_hosszu(file = "DH_nagy.xlsx", nevek = pair_list, val = "tHb (g/dL)")

y_mosm = df_mosm_hosszu.iloc[:,1:11].fillna(df_mosm_hosszu.iloc[:,1:].median()).apply(lambda x: np.median(x), axis=0)
x_mosm = df_mosm_hosszu.iloc[:,1:11].columns
y_hgb_h = df_hgb_hosszu.iloc[:,1:11].fillna(df_hgb_hosszu.iloc[:,1:].median()).apply(lambda x: np.median(x), axis=0)
x_hgb_h = df_hgb_hosszu.iloc[:,1:11].columns

df_mosm_hosszu_long = df_mosm_hosszu.iloc[:,0:11].melt()
df_mosm_hosszu_long = df_mosm_hosszu_long.loc[df_mosm_hosszu_long.variable != "Minta",:]

df_hgb_h_long = df_hgb_hosszu.iloc[:,0:11].melt()
df_hgb_h_long = df_hgb_h_long.loc[df_hgb_h_long.variable != "Minta",:]


In [18]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from itertools import cycle

df_mosm_hosszu_long = df_mosm_hosszu_long.dropna()
df_hgb_h_long = df_hgb_h_long.dropna()


palette = cycle(px.colors.qualitative.Antique)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x_hgb_h, y=y_hgb_h,
    name='HGB line', marker_color=next(palette))
)
fig.add_trace(go.Box(
    x=df_hgb_h_long.variable, y=df_hgb_h_long.value,
    name='HGB box', marker_color=next(palette))
)


fig.add_trace(go.Scatter(
    x=x_mosm, y=y_mosm,
    name='mOsm line', marker_color=next(palette))
)
fig.add_trace(go.Box(
    x=df_mosm_hosszu_long.variable, y=df_mosm_hosszu_long.value,
    name='mOsm box', marker_color=next(palette))
)

fig.update_layout(font_family='Times New Roman', showlegend=True)
fig.update_layout(height=600, width=800, title_text="")
fig.show()

In [25]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from itertools import cycle

df_mosm_hosszu_long = df_mosm_hosszu_long.dropna()
df_hgb_h_long = df_hgb_h_long.dropna()

df_mosm_hosszu_long.variable = df_mosm_hosszu_long.variable.replace({'Nyugalmi (T. előtt)':"Rest", 'VM':"Max"}) 
df_hgb_h_long.variable = df_hgb_h_long.variable.replace({'Nyugalmi (T. előtt)':"Rest", 'VM':"Max"})

x_hgb_h = df_hgb_h_long.variable.unique()
x_mosm = df_hgb_h_long.variable.unique()

palette = cycle(px.colors.qualitative.Antique)

hgb_max = np.max(y_hgb_h)
mosm_max = np.max(y_mosm)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x_hgb_h, y=y_hgb_h/hgb_max,
    name='HGB line', marker_color=next(palette))
)
fig.add_trace(go.Box(
    x=df_hgb_h_long.variable, y=df_hgb_h_long.value/hgb_max,
    name='HGB box', marker_color=next(palette))
)


fig.add_trace(go.Scatter(
    x=x_mosm, y=y_mosm/mosm_max,
    name='mOsm line', marker_color=next(palette))
)
fig.add_trace(go.Box(
    x=df_mosm_hosszu_long.variable, y=df_mosm_hosszu_long.value/mosm_max,
    name='mOsm box', marker_color=next(palette))
)

fig.update_layout(font_family='Times New Roman', showlegend=True)
fig.update_layout(height=600, width=800, title_text="")
fig.show()

In [22]:
df_hgb_h_long.variable.unique()


array(['Nyugalmi (T. előtt)', "20'", "40'", "60'", "80'", "100'", "120'",
       'VM', "R5'", "R30'"], dtype=object)